In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub
import os
import glob
import random
from sklearn.metrics import confusion_matrix, classification_report

2.19.0


In [ ]:
path = kagglehub.dataset_download("iarunava/cell-images-for-detecting-malaria")
path = path + '/cell_images/cell_images'
print("Path to dataset files:", path)

In [ ]:
list_dir = os.listdir(path)
Parasitized_dir = glob.glob(os.path.join(path, 'Parasitized', '*'))
Uninfected_dir = glob.glob(os.path.join(path, 'Uninfected', '*'))

Para_len, Uninf_len = len(Parasitized_dir)-1, len(Uninfected_dir)-1

plt.figure(figsize=(10, 10))
for i in range(1,6):
    plt.subplot(5, 5, i)
    plt.grid(False)
    plt.axis("off")
    plt.imshow(plt.imread(Parasitized_dir[random.randint(0, Para_len)]),cmap=plt.cm.binary)
    plt.title("Parasited")

plt.figure(figsize=(10, 10))
for i in range(1,6):
    plt.subplot(5, 5, i)
    plt.grid(False)
    plt.axis("off")
    plt.imshow(plt.imread(Uninfected_dir[random.randint(0, Uninf_len)]),cmap=plt.cm.binary)
    plt.title("Uninfected")


In [ ]:
data_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,height_shift_range=0.2,
                              shear_range=0.2, zoom_range=0.2, horizontal_flip=True, vertical_flip=True, validation_split=0.2)

train_data = data_gen.flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode='binary',
                                          shuffle=True, seed=3199, subset="training")

val_data = data_gen.flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode='binary',
                                          shuffle=False, seed=3199, subset="validation")

class_names = train_data.class_indices

In [ ]:
BaseModel = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

BaseModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ModelHistory = BaseModel.fit(train_data, epochs=5, validation_data=val_data)

BaseModel.summary()

In [ ]:
img_dir = Parasitized_dir[random.randint(0, Para_len)]
img = tf.keras.utils.load_img(img_dir, target_size=(224,224))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

save_path = "MyBaseModel.keras"
if not os.path.exists(save_path):
    BaseModel.save(save_path)
    print(f"Model saved at: {save_path}")
else:
    print("Model already exists.... Skip saving... \n\n")

LoadBaseModel = tf.keras.models.load_model(save_path)

prediction = BaseModel.predict(img_array)
for i in class_names.keys():
  if prediction[0][0] == class_names[i]:
    print("Prediction: ", i)
    plt.figure(figsize=(3,3))
    plt.imshow(plt.imread(img_dir),cmap=plt.cm.binary)
    plt.grid(False)
    plt.title(i)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(ModelHistory.history["accuracy"], marker="o", label="Accuracy", )
plt.plot(ModelHistory.history["val_accuracy"], marker="o", label="Validation Accuracy")
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()

plt.subplot(1,2,2)
plt.plot(ModelHistory.history["loss"], marker="o", label="Losses")
plt.plot(ModelHistory.history["val_loss"], marker="o", label="Validation Losses")
plt.title("Model Losses")
plt.ylabel("Lose")
plt.xlabel("Epoch")
plt.legend()

loss, accuracy = BaseModel.evaluate(val_data)
print("Loss: ", loss, "\nAccuracy: ", accuracy)
y_prediction = BaseModel.predict(val_data)
y_true = val_data.classes

In [ ]:
cm = confusion_matrix(y_true, y_prediction.round())
plt.figure(figsize= (10, 10))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

classes = list(class_names.keys())
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')
print(classification_report(y_true, y_prediction.round(), target_names= classes ))